### Learning AutoGen RoundRobin Group Chat

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console

from dotenv import load_dotenv

import os

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")
azure_openai_deployment_name = os.getenv(
    "AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo")

if not azure_openai_api_key or \
        not azure_openai_endpoint or \
        not azure_openai_api_version or \
        not azure_openai_deployment_name:
    raise ValueError(
        "Please set Azure Open AI Endpoint Details in your environment variables.")

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
blog_post_draft_agent = AssistantAgent(
    name="BlogPostDraftAgent",
    model_client=model_client,
    system_message="""
    You are an expert blog post writer. Your task is to draft a blog post based on the provided topic.
    The blog post should be engaging, informative, and well-structured.
    Please ensure that the draft is coherent and flows logically from one section to another.
    """,
)

blog_post_enhancement_agent = AssistantAgent(
    name="BlogPostEnhancementAgent",
    model_client=model_client,
    system_message="""
        You are an expert editor. 
        Your task is to enhance the blog post draft by adding more details and improving the language.
        If you're satisfied with the draft, you can finalize it by saying 'FINAL'.
        """,
)

In [ ]:
text_mention_condition = TextMentionTermination("FINAL")

team = RoundRobinGroupChat(
    [blog_post_draft_agent, blog_post_enhancement_agent],
    termination_condition=text_mention_condition,
)

In [ ]:
await Console(
    team.run_stream(task="Create a short blog post about the benefits of meditation."))